In [ ]:
import os
import pandas as pd
import fetchData
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset_purposes = ['train', 'test']

FUSER_airports = ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']

file_types = ['first_position', 'ETD', 'runways', 'LAMP', 'TBFM', 'TFM_track', 'configs',  'MFS']

years = ['2022', '2023']
monthes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', 
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', 
       '25', '26', '27', '28', '29', '30', '31']

In [3]:
data_type = 'FUSER'

In [13]:
for dataset_purpose in dataset_purposes:
    for year in years:
        for month in monthes:
            for day in days:
                data_range = f"{year}-{month}-{day}"

                df_dic = {}
                for file_type in file_types:
                    data_list = []
                    for airport in FUSER_airports:
                        file_name = fetchData.get_fuser_file_name(
                                airport=airport,
                                data_range=data_range,
                                file_type=file_type
                            )
                        output_file_name = f"{data_range}"
                 
                        if not fetchData.check_output_file_exists(data_type=data_type,
                            dataset_purpose=dataset_purpose,
                            file_name=output_file_name,
                            base_dir='./data') and fetchData.check_input_file_exists(data_type=data_type,
                            dataset_purpose=dataset_purpose,
                            path_level=airport,
                            file_name=file_name):
                            data=fetchData.load_data(
                                data_type=data_type,
                                dataset_purpose=dataset_purpose,
                                path_level=airport,
                                file_name=file_name)
                            
                            if 'airport_id' not in data.columns:
                                data['airport_id'] = airport
                            data_list.append(data)
                        
                    start_date = None
                    end_date = None
                    complete_times = None
                    complete_airports = None
                    complete_index = None
                    complete_df = None
                    df = None
                    if data_list:
                        df = pd.concat(data_list, ignore_index=True)
                        start_date = pd.to_datetime(data_range).tz_localize('UTC')
                        end_date = start_date + pd.Timedelta(days=1)
                        complete_times = pd.date_range(start=start_date, end=end_date, freq='15min')
                        complete_airports = FUSER_airports.copy()

                        complete_index = pd.MultiIndex.from_product([complete_times, complete_airports], names=['timestamp_15mins', 'airport_id'])
                        complete_df = pd.DataFrame(index=complete_index).reset_index()

                    if df is not None and not df.empty and file_type == 'first_position':                        
                        df['time_first_tracked'] = pd.to_datetime(df['time_first_tracked'], utc=True)
                        df['timestamp_15mins'] = df['time_first_tracked'].dt.floor('15min')
                        
                        first_tracked_15mins_counts = df.groupby(['timestamp_15mins', 'airport_id'])['gufi'].count().reset_index()
                        first_tracked_15mins_counts.rename(columns={'gufi': 'flights_tracked_count'}, inplace=True)
                        
                        df_aligned = complete_df.merge(first_tracked_15mins_counts, on=['timestamp_15mins', 'airport_id'], how='left')
                        numeric_cols = df_aligned.select_dtypes(include=[np.number]).columns                 
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].ffill().bfill()
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].interpolate(method='linear', limit_direction='both')
                        df_dic[file_type] = df_aligned
                    elif df is not None and not df.empty and file_type == 'ETD':
                        df['departure_runway_estimated_time'] = pd.to_datetime(df['departure_runway_estimated_time'], utc=True)
                        df['timestamp_15mins'] = df['departure_runway_estimated_time'].dt.floor('15min')
                        
                        ETD_15min_counts = df.groupby(['timestamp_15mins', 'airport_id'])['gufi'].count().reset_index()
                        ETD_15min_counts.rename(columns={'gufi': 'ETD_estimated_time_count'}, inplace=True)
                        
                        df_aligned = complete_df.merge(ETD_15min_counts, on=['timestamp_15mins', 'airport_id'], how='left')
                        numeric_cols = df_aligned.select_dtypes(include=[np.number]).columns                 
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].ffill()
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].interpolate(method='linear', limit_direction='both')
                        df_dic[file_type] = df_aligned
                    elif df is not None and not df.empty and file_type == 'runways':
                        df['arrival_runway_actual_time'] = pd.to_datetime(df['arrival_runway_actual_time'], utc=True)
                        df['timestamp_15mins'] = df['arrival_runway_actual_time'].dt.floor('15min')
                        

                        runway_arrival_15min_counts = df.groupby(['timestamp_15mins', 'airport_id'])['gufi'].count().reset_index()
                        runway_arrival_15min_counts.rename(columns={'gufi': 'runway_estimated_time_count'}, inplace=True)
                        
                        df_aligned = complete_df.merge(runway_arrival_15min_counts, on=['timestamp_15mins', 'airport_id'], how='left')
                        numeric_cols = df_aligned.select_dtypes(include=[np.number]).columns                 
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].ffill()
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].interpolate(method='linear', limit_direction='both')
                        df_dic[file_type] = df_aligned
                    elif df is not None and not df.empty and file_type == 'MFS':
                        time_columns = ['arrival_stand_actual_time', 'arrival_runway_actual_time', 
                                        'departure_stand_actual_time', 'departure_runway_actual_time']
                        
                        for col in time_columns:
                            df[col] = pd.to_datetime(df[col], errors='coerce')
                            df[col] = df[col].dt.tz_localize('utc')
                            df[f'{col}_15mins'] = df[col].dt.floor('15min')
                        
                        # Calculate residence time
                        df['turnaround_time'] = (df['departure_stand_actual_time'] - df['arrival_stand_actual_time']).dt.total_seconds() / 60  # 转换为分钟
                        
                        # Create a uniform timestamp column `timestamp_15mins` taht averages arrival and departure times and rounds down to15 mins.
                        df['timestamp_15mins'] = (
                            df[['arrival_runway_actual_time', 'departure_runway_actual_time']]
                            .mean(axis=1)
                            .dt.floor('15min')
                        )
                        
                        # Converts isarrival and isdepartureto numeric types C
                        df['isarrival'] = df['isarrival'].astype(int)
                        df['isdeparture'] = df['isdeparture'].astype(int)

                        df['gufi_prefix'] = df['gufi'].str.extract(r'^([A-Za-z]+)')
                        df['major_carrier'] = df['major_carrier'].fillna(df['gufi_prefix'])
                        df.drop(columns=['gufi_prefix'])

                        # Coded categorical variables
                        df = pd.get_dummies(df, columns=['major_carrier', 'aircraft_type', 'aircraft_engine_class', 'flight_type'])                       
                        
                        grouped_df = df.groupby(['airport_id', 'timestamp_15mins']).agg({
                            'isarrival': 'sum',                   
                            'isdeparture': 'sum',                 
                            'turnaround_time': 'mean',  
                            **{col: 'max' for col in df.columns if col.startswith('major_carrier_')}, 
                            **{col: 'max' for col in df.columns if col.startswith('aircraft_type_')}, 
                            **{col: 'max' for col in df.columns if col.startswith('aircraft_engine_class_')}, 
                            **{col: 'max' for col in df.columns if col.startswith('flight_type_')},         
                        }).reset_index()
                        
                        grouped_df.rename(columns={'isarrival': 'arrival_count', 'isdeparture': 'departure_count', 
                                                'turnaround_time': 'avg_turnaround_time'}, inplace=True)
                        
                        df_aligned = complete_df.merge(grouped_df, on=['timestamp_15mins', 'airport_id'], how='left')
                    
                        numeric_cols = df_aligned.select_dtypes(include=[np.number]).columns                 
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].ffill()
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].interpolate(method='linear', limit_direction='both')

                        object_columns = df_aligned.select_dtypes(include=['object']).columns
                        object_columns = [col for col in object_columns if col !='airport_id']

                        for col in object_columns:
                            df_aligned[col] = df_aligned[col].map({True: True, False: False, 'True': True, 'False': False, np.nan: np.nan})

                        df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()
                        df_aligned[object_columns] = df_aligned[object_columns].infer_objects(copy=False)
                        nan_s = df_aligned.isnull().sum()
                        df_dic[file_type] = df_aligned
                    elif df is not None and not df.empty and file_type == 'TBFM':
                        df['arrival_runway_sta'] = pd.to_datetime(df['arrival_runway_sta'], utc=True)
                        df['timestamp_15mins'] = df['arrival_runway_sta'].dt.floor('15min')

                        TBFM_scheduled_arrivals_15min_counts = df.groupby(['timestamp_15mins', 'airport_id'])['gufi'].count().reset_index()
                        TBFM_scheduled_arrivals_15min_counts.rename(columns={'gufi': 'scheduled_arrivals_count'}, inplace=True)
                        
                        df_aligned = complete_df.merge(TBFM_scheduled_arrivals_15min_counts, on=['timestamp_15mins', 'airport_id'], how='left')
                        numeric_cols = df_aligned.select_dtypes(include=[np.number]).columns 
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].ffill()
                        df_dic[file_type] = df_aligned
                    elif df is not None and not df.empty and file_type == 'TFM_track':
                        df['arrival_runway_estimated_time'] = pd.to_datetime(df['arrival_runway_estimated_time'], utc=True)
                        df['timestamp_15mins'] = df['arrival_runway_estimated_time'].dt.floor('15min')
                        
                        TFM_estimated_arrivals_15min_counts = df.groupby(['timestamp_15mins', 'airport_id'])['gufi'].count().reset_index()
                        TFM_estimated_arrivals_15min_counts.rename(columns={'gufi': 'estimated_arrivals_count'}, inplace=True)
                        
                        df_aligned = complete_df.merge(TFM_estimated_arrivals_15min_counts, on=['timestamp_15mins', 'airport_id'], how='left')
                        
                        numeric_cols = df_aligned.select_dtypes(include=[np.number]).columns                 
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].ffill()
                        
                        df_dic[file_type] = df_aligned
                    elif df is not None and not df.empty and file_type == 'configs':
                        df['start_time'] = pd.to_datetime(df['start_time'], utc=True)
                        df['timestamp_15mins'] = df['start_time'].dt.floor('15min')

                        # Split runways into list and create binary indicator
                        df['arrival_runways'] = df['arrival_runways'].fillna('').apply(lambda x: x.split(','))
                        df['departure_runways'] = df['departure_runways'].fillna('').apply(lambda x: x.split(','))
                        
                        # Explicitly create runway indicator columns
                        arrival_runways = df['arrival_runways'].explode().unique()
                        departure_runways = df['departure_runways'].explode().unique()

                        # Encode each runway as a binary feature
                        for runway in arrival_runways:
                            if runway:  # Avoid empty strings
                                df[f'arrival_runway_{runway.strip()}'] = df['arrival_runways'].apply(lambda x: 1 if runway.strip() in x else 0)

                        for runway in departure_runways:
                            if runway:  # Avoid empty strings
                                df[f'departure_runway_{runway.strip()}'] = df['departure_runways'].apply(lambda x: 1 if runway.strip() in x else 0)

                        # Aggregate by 15-minute intervals to get runway and configuration features
                        configs_15min = df.groupby(['timestamp_15mins', 'airport_id']).agg({
                            # Runway availability indicators
                            **{f'arrival_runway_{runway.strip()}': 'max' for runway in arrival_runways if runway},
                            **{f'departure_runway_{runway.strip()}': 'max' for runway in departure_runways if runway}
                        }).reset_index()

                        df_aligned = complete_df.merge(configs_15min, on=['timestamp_15mins', 'airport_id'], how='left')
                        numeric_cols = df_aligned.select_dtypes(include=[np.number]).columns                 
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].ffill().bfill()
                        #df_aligned[numeric_cols] = df_aligned[numeric_cols].interpolate(method='linear', limit_direction='both')
                        df_dic[file_type] = df_aligned
                    elif df is not None and not df.empty and file_type == 'LAMP':
                        df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
                        df['timestamp_15mins'] = df['timestamp'].dt.floor('30min')
                        
                        # Define acceptable ranges for temperature and wind speed
                        TEMPERATURE_MIN, TEMPERATURE_MAX = -50, 130  # Example realistic range for temperature (Celsius)
                        WIND_SPEED_MAX = 12  # Example max realistic wind speed (m/s)

                        # Step 1: Replace unrealistic values with NaN for better handling
                        # Replace temperature values outside the range with NaN
                        df['temperature'] = df['temperature'].apply(lambda x: x if TEMPERATURE_MIN <= x <= TEMPERATURE_MAX else np.nan)

                        # Replace wind speed values that are negative or exceed the max realistic value
                        df['wind_speed'] = df['wind_speed'].apply(lambda x: x if 0 <= x <= WIND_SPEED_MAX else np.nan)
                        df['wind_gust'] = df['wind_gust'].apply(lambda x: x if 0 <= x <= WIND_SPEED_MAX else np.nan)

                        # Step 2: Check and handle missing values (after marking unrealistic values as NaN)

                        # Fill missing values for temperature and wind speed using interpolation and forward/backward fill
                        df['temperature'] = df['temperature'].interpolate(method='linear').ffill().bfill()
                        df['wind_speed'] = df['wind_speed'].interpolate(method='linear').ffill().bfill()

                        df = pd.get_dummies(df, columns=['cloud', 'lightning_prob'], prefix=['cloud', 'lightning_prob'])

                        # Define aggregation methods
                        LAMP_15min = df.groupby(['timestamp_15mins', 'airport_id']).agg({
                            'temperature': 'mean',
                            'wind_direction': 'mean',
                            'wind_speed': 'mean',
                            'wind_gust': 'mean',
                            'cloud_ceiling': 'mean',
                            'visibility': 'mean',
                            **{col: 'max' for col in df.columns if col.startswith('cloud_')},# Most frequent cloud type
                            **{col: 'max' for col in df.columns if col.startswith('lightning_prob_')}, # Most frequent lightning probability  
                            'precip': 'max'  # Any forecast of precipitation within the interval
                        }).reset_index()

                        LAMP_15min['temperature'] = LAMP_15min['temperature'].replace(0, np.nan)

                        LAMP_15min['wind_speed'] = LAMP_15min['wind_speed'].replace(0, np.nan)
                        LAMP_15min['wind_speed'] = LAMP_15min['wind_speed'].interpolate(method='linear').bfill().ffill()

                        df_aligned = complete_df.merge(LAMP_15min, on=['timestamp_15mins', 'airport_id'], how='left')
                        
                        numeric_cols = df_aligned.select_dtypes(include=[np.number]).columns                 
                        #df_aligned[numeric_cols] = df_aligned[numeric_cols].ffill()
                        
                        df_aligned[numeric_cols] = df_aligned[numeric_cols].interpolate(method='linear', limit_direction='both')

                        
                        object_columns = df_aligned.select_dtypes(include=['object']).columns
                        object_columns = [col for col in object_columns if col !='airport_id']


                        for col in object_columns:
                            df_aligned[col] = df_aligned[col].map({True: True, False: False, 'True': True, 'False': False, np.nan: np.nan})

                        df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()
                        df_aligned[object_columns] = df_aligned[object_columns].infer_objects(copy=False)
                        df_dic[file_type] = df_aligned



                if df_dic:
                    keys = list(df_dic.keys())
                    merged_df = df_dic[keys[0]]
                    
                    for key in keys[1:]:
                        if key != 'TBFM':  
                            df = df_dic[key]
                            
                            merged_df = pd.merge(
                                merged_df,
                                df,
                                on=['timestamp_15mins', 'airport_id'],  
                                how='outer'
                            )


                    file_path = f"./data/{data_type}/{dataset_purpose}/{data_range}.csv"
                    os.makedirs(os.path.dirname(file_path), exist_ok=True)
                    merged_df.to_csv(file_path, index=False)
                    print(file_path, "saved!")


/tmp/ipykernel_71111/4004501272.py:262: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()
/tmp/ipykernel_71111/4004501272.py:144: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()


./data/FUSER/train/2022-12-29.csv saved!


/tmp/ipykernel_71111/4004501272.py:262: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()
/tmp/ipykernel_71111/4004501272.py:144: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()


./data/FUSER/train/2023-01-07.csv saved!


/tmp/ipykernel_71111/4004501272.py:262: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()
/tmp/ipykernel_71111/4004501272.py:144: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()


...

./data/FUSER/test/2023-08-16.csv saved!


/tmp/ipykernel_71111/4004501272.py:262: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()
/tmp/ipykernel_71111/4004501272.py:144: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()


./data/FUSER/test/2023-08-17.csv saved!


/tmp/ipykernel_71111/4004501272.py:262: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()
/tmp/ipykernel_71111/4004501272.py:144: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_aligned[object_columns] = df_aligned[object_columns].ffill().bfill()


./data/FUSER/test/2023-08-18.csv saved!
